In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

# Get train, test, and val paths of images
This code relies on the notebook being started from a bash shell

In [2]:
train_viral = !find 'data/chest_xray/train/PNEUMONIA/' -regex '.*virus.*'
train_bacterial = !find 'data/chest_xray/train/PNEUMONIA/' -regex '.*bacteria.*'
train_normal = !ls 'data/chest_xray/train/NORMAL'
train_normal = ['data/chest_xray/train/NORMAL/' + file for file in train_normal]

test_viral = !find 'data/chest_xray/test/PNEUMONIA/' -regex '.*virus.*'
test_bacterial = !find 'data/chest_xray/test/PNEUMONIA/' -regex '.*bacteria.*'
test_normal = !ls 'data/chest_xray/test/NORMAL'
test_normal = ['data/chest_xray/test/NORMAL/' + file for file in test_normal]

val_viral = !find 'data/chest_xray/val/PNEUMONIA/' -regex '.*virus.*'
val_bacterial = !find 'data/chest_xray/val/PNEUMONIA/' -regex '.*bacteria.*'
val_normal = !ls 'data/chest_xray/val/NORMAL'
val_normal = ['data/chest_xray/val/NORMAL/' + file for file in val_normal]

Code from Grace's EDA notebook

In [7]:
# Creating an datagenerator object that will perform image augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=90,
                                   width_shift_range=.2,
                                   height_shift_range=.2,
                                   shear_range=.2,
                                   zoom_range=.2,
                                   horizontal_flip=True,
                                   brightness_range=[.5, 1.5])

# Datagenerators for the test and validation set will only rescale the images
test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)


# Applying the datagenerator objects to the images in the folders
image_size = (96, 96)
training_set = train_datagen.flow_from_directory('data/chest_xray/train',
                                                 target_size=image_size,
                                                 batch_size=32,
                                                 class_mode='binary',
                                                 color_mode='grayscale')

val_set = val_datagen.flow_from_directory('data/chest_xray/val',
                                          target_size=image_size,
                                          batch_size=32,
                                          class_mode='binary',
                                          color_mode='grayscale')

test_set = test_datagen.flow_from_directory('data/chest_xray/test',
                                            target_size=image_size,
                                            batch_size=32,
                                            class_mode='binary', 
                                            color_mode='grayscale')

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [4]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(96, 96, 1)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(32, (4, 4), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['acc'])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 94, 94, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 47, 47, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 44, 44, 32)        16416     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 22, 22, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6400)              0

In [9]:
results = model.fit(training_set, batch_size=32, validation_data=val_set, epochs=15)

Epoch 1/15
163/163 [==============================] - 146s 898ms/step - loss: 0.5041 - acc: 0.7398 - val_loss: 0.8643 - val_acc: 0.5000
Epoch 2/15
163/163 [==============================] - 109s 671ms/step - loss: 0.4681 - acc: 0.7427 - val_loss: 0.7270 - val_acc: 0.5000
Epoch 3/15
163/163 [==============================] - 101s 617ms/step - loss: 0.4501 - acc: 0.7636 - val_loss: 0.7117 - val_acc: 0.5625
Epoch 4/15
163/163 [==============================] - 97s 596ms/step - loss: 0.4262 - acc: 0.7905 - val_loss: 0.7012 - val_acc: 0.8125
Epoch 5/15
163/163 [==============================] - 104s 637ms/step - loss: 0.3909 - acc: 0.8194 - val_loss: 0.6520 - val_acc: 0.8125
Epoch 6/15
163/163 [==============================] - 93s 568ms/step - loss: 0.3441 - acc: 0.8497 - val_loss: 0.5233 - val_acc: 0.8125
Epoch 7/15
163/163 [==============================] - 92s 565ms/step - loss: 0.3287 - acc: 0.8560 - val_loss: 0.7220 - val_acc: 0.7500
Epoch 8/15
163/163 [==============================]